In [2]:
# import lib
import argparse, math, sys
from copy import deepcopy
import torch, torchvision
from torch import nn
import torch.nn.functional as F

if torch.cuda.is_available():
    torch.backends.cudnn.benchmark = True
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

In [16]:
# declare functions

def entropy(target):
    probas = []
    for k in range(target.max() + 1):
        n = (target == k).sum().item()
        if n > 0: probas.append(n)
    probas = torch.tensor(probas).float()
    probas /= probas.sum()
    return - (probas * probas.log()).sum().item()

# Returns a triplet of tensors (a, b, c), where a and b contain each
# half of the samples, with a[i] and b[i] of same class for any i, and
# c is a 1d long tensor real classes
def create_image_pairs(train = False):
    ua, ub, uc = [], [], []

    if train: input, target = train_input, train_target
    else:     input, target = test_input, test_target

    for i in used_MNIST_classes:
        used_indices = torch.arange(input.size(0), device = target.device).masked_select(target == i.item())
        x = input[used_indices]
        x = x[torch.randperm(x.size(0))]
        hs = x.size(0)//2
        ua.append(x.narrow(0, 0, hs))
        ub.append(x.narrow(0, hs, hs))
        uc.append(target[used_indices])

    a = torch.cat(ua, 0)
    b = torch.cat(ub, 0)
    c = torch.cat(uc, 0)
    perm = torch.randperm(a.size(0))
    a = a[perm].contiguous()

    # if args.independent: perm = torch.randperm(a.size(0))
    b = b[perm].contiguous()

    return a, b, c

# Returns a triplet a, b, c where a are the standard MNIST images, c
# the classes, and b is a Nx2 tensor, with for every n:
#   b[n, 0] ~ Uniform(0, 10)
#   b[n, 1] ~ b[n, 0] + Uniform(0, 0.5) + c[n]
def create_image_values_pairs(train = False):
    ua, ub = [], []

    if train:
        input, target = train_input, train_target
    else:
        input, target = test_input, test_target

    m = torch.zeros(used_MNIST_classes.max() + 1, dtype = torch.uint8, device = target.device)
    m[used_MNIST_classes] = 1
    m = m[target]
    used_indices = torch.arange(input.size(0), device = target.device).masked_select(m)

    input = input[used_indices].contiguous()
    target = target[used_indices].contiguous()

    a = input
    c = target

    b = a.new(a.size(0), 2)
    b[:, 0].uniform_(0.0, 10.0)
    b[:, 1].uniform_(0.0, 0.5)

    if args.independent:
        b[:, 1] += b[:, 0] + \
                   used_MNIST_classes[torch.randint(len(used_MNIST_classes), target.size())]
    else:
        b[:, 1] += b[:, 0] + target.float()

    return a, b, c

def create_sequences_pairs(train = False):
    nb, length = 10000, 1024
    noise_level = 2e-2

    ha = torch.randint(args.nb_classes, (nb, ), device = device) + 1
    if args.independent:
        hb = torch.randint(args.nb_classes, (nb, ), device = device)
    else:
        hb = ha

    pos = torch.empty(nb, device = device).uniform_(0.0, 0.9)
    a = torch.linspace(0, 1, length, device = device).view(1, -1).expand(nb, -1)
    a = a - pos.view(nb, 1)
    a = (a >= 0).float() * torch.exp(-a * math.log(2) / 0.1)
    a = a * ha.float().view(-1, 1).expand_as(a) / (1 + args.nb_classes)
    noise = a.new(a.size()).normal_(0, noise_level)
    a = a + noise

    pos = torch.empty(nb, device = device).uniform_(0.0, 0.5)
    b1 = torch.linspace(0, 1, length, device = device).view(1, -1).expand(nb, -1)
    b1 = b1 - pos.view(nb, 1)
    b1 = (b1 >= 0).float() * torch.exp(-b1 * math.log(2) / 0.1) * 0.25
    pos = pos + hb.float() / (args.nb_classes + 1) * 0.5
    # pos += pos.new(hb.size()).uniform_(0.0, 0.01)
    b2 = torch.linspace(0, 1, length, device = device).view(1, -1).expand(nb, -1)
    b2 = b2 - pos.view(nb, 1)
    b2 = (b2 >= 0).float() * torch.exp(-b2 * math.log(2) / 0.1) * 0.25

    b = b1 + b2
    noise = b.new(b.size()).normal_(0, noise_level)
    b = b + noise

    return a, b, ha

class NetForImagePair(nn.Module):
    def __init__(self):
        super(NetForImagePair, self).__init__()
        self.features_a = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size = 5),
            nn.MaxPool2d(3), nn.ReLU(),
            nn.Conv2d(16, 32, kernel_size = 5),
            nn.MaxPool2d(2), nn.ReLU(),
        )

        self.features_b = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size = 5),
            nn.MaxPool2d(3), nn.ReLU(),
            nn.Conv2d(16, 32, kernel_size = 5),
            nn.MaxPool2d(2), nn.ReLU(),
        )

        self.fully_connected = nn.Sequential(
            nn.Linear(256, 200),
            nn.ReLU(),
            nn.Linear(200, 1)
        )

    def forward(self, a, b):
        a = self.features_a(a).view(a.size(0), -1)
        b = self.features_b(b).view(b.size(0), -1)
        x = torch.cat((a, b), 1)
        return self.fully_connected(x)

class NetForImageValuesPair(nn.Module):
    def __init__(self):
        super(NetForImageValuesPair, self).__init__()
        self.features_a = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size = 5),
            nn.MaxPool2d(3), nn.ReLU(),
            nn.Conv2d(16, 32, kernel_size = 5),
            nn.MaxPool2d(2), nn.ReLU(),
        )

        self.features_b = nn.Sequential(
            nn.Linear(2, 32), nn.ReLU(),
            nn.Linear(32, 32), nn.ReLU(),
            nn.Linear(32, 128), nn.ReLU(),
        )

        self.fully_connected = nn.Sequential(
            nn.Linear(256, 200),
            nn.ReLU(),
            nn.Linear(200, 1)
        )

    def forward(self, a, b):
        a = self.features_a(a).view(a.size(0), -1)
        b = self.features_b(b).view(b.size(0), -1)
        x = torch.cat((a, b), 1)
        return self.fully_connected(x)

class NetForSequencePair(nn.Module):

    def feature_model(self):
        kernel_size = 11
        pooling_size = 4
        return  nn.Sequential(
            nn.Conv1d(      1, self.nc, kernel_size = kernel_size),
            nn.AvgPool1d(pooling_size),
            nn.LeakyReLU(),
            nn.Conv1d(self.nc, self.nc, kernel_size = kernel_size),
            nn.AvgPool1d(pooling_size),
            nn.LeakyReLU(),
            nn.Conv1d(self.nc, self.nc, kernel_size = kernel_size),
            nn.AvgPool1d(pooling_size),
            nn.LeakyReLU(),
            nn.Conv1d(self.nc, self.nc, kernel_size = kernel_size),
            nn.AvgPool1d(pooling_size),
            nn.LeakyReLU(),
        )

    def __init__(self):
        super(NetForSequencePair, self).__init__()

        self.nc = 32
        self.nh = 256

        self.features_a = self.feature_model()
        self.features_b = self.feature_model()

        self.fully_connected = nn.Sequential(
            nn.Linear(2 * self.nc, self.nh),
            nn.ReLU(),
            nn.Linear(self.nh, 1)
        )

    def forward(self, a, b):
        a = a.view(a.size(0), 1, a.size(1))
        a = self.features_a(a)
        a = F.avg_pool1d(a, a.size(2))

        b = b.view(b.size(0), 1, b.size(1))
        b = self.features_b(b)
        b = F.avg_pool1d(b, b.size(2))

        x = torch.cat((a.view(a.size(0), -1), b.view(b.size(0), -1)), 1)
        return self.fully_connected(x)

In [8]:
# read the data
torch.manual_seed(567)
used_MNIST_classes = torch.tensor(eval('[' + "1,2,3" + ']'), device = device)
print(used_MNIST_classes)

train_set = torchvision.datasets.MNIST('./data/mnist/', train = True, download = True)
train_input  = train_set.train_data.view(-1, 1, 28, 28).to(device).float()
train_target = train_set.train_labels.to(device)

test_set = torchvision.datasets.MNIST('./data/mnist/', train = False, download = True)
test_input  = test_set.test_data.view(-1, 1, 28, 28).to(device).float()
test_target = test_set.test_labels.to(device)

mu, std = train_input.mean(), train_input.std()
train_input.sub_(mu).div_(std)
print(test_input.sub_(mu).div_(std))

tensor([1, 2, 3])
tensor([[[[-0.4240, -0.4240, -0.4240,  ..., -0.4240, -0.4240, -0.4240],
          [-0.4240, -0.4240, -0.4240,  ..., -0.4240, -0.4240, -0.4240],
          [-0.4240, -0.4240, -0.4240,  ..., -0.4240, -0.4240, -0.4240],
          ...,
          [-0.4240, -0.4240, -0.4240,  ..., -0.4240, -0.4240, -0.4240],
          [-0.4240, -0.4240, -0.4240,  ..., -0.4240, -0.4240, -0.4240],
          [-0.4240, -0.4240, -0.4240,  ..., -0.4240, -0.4240, -0.4240]]],


        [[[-0.4240, -0.4240, -0.4240,  ..., -0.4240, -0.4240, -0.4240],
          [-0.4240, -0.4240, -0.4240,  ..., -0.4240, -0.4240, -0.4240],
          [-0.4240, -0.4240, -0.4240,  ..., -0.4240, -0.4240, -0.4240],
          ...,
          [-0.4240, -0.4240, -0.4240,  ..., -0.4240, -0.4240, -0.4240],
          [-0.4240, -0.4240, -0.4240,  ..., -0.4240, -0.4240, -0.4240],
          [-0.4240, -0.4240, -0.4240,  ..., -0.4240, -0.4240, -0.4240]]],


        [[[-0.4240, -0.4240, -0.4240,  ..., -0.4240, -0.4240, -0.4240],
        

In [21]:
# create pair and train
# if args.data == 'image_pair':
#     create_pairs = create_image_pairs
#     model = NetForImagePair()
# elif args.data == 'image_values_pair':
#     create_pairs = create_image_values_pairs
#     model = NetForImageValuesPair()
# elif args.data == 'sequence_pair':
#     create_pairs = create_sequences_pairs
#     model = NetForSequencePair()

#     ######################
#     ## Save for figures
#     a, b, c = create_pairs()
#     for k in range(10):
#         file = open(f'train_{k:02d}.dat', 'w')
#         for i in range(a.size(1)):
#             file.write(f'{a[k, i]:f} {b[k,i]:f}\n')
#         file.close()

create_pairs = create_image_pairs
model = NetForImagePair()
nb_epochs = 50
learning_rate = 0.0008
batch_size = 20

######################################################################
# Train

print(f'nb_parameters {sum(x.numel() for x in model.parameters())}')

model.to(device)

input_a, input_b, classes = create_pairs(train = True)

for e in range(nb_epochs):

    optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

    input_br = input_b[torch.randperm(input_b.size(0))]

    acc_mi = 0.0

    for batch_a, batch_b, batch_br in zip(input_a.split(batch_size),input_b.split(batch_size),input_br.split(batch_size)):
        mi = model(batch_a, batch_b).mean() - model(batch_a, batch_br).exp().mean().log()
        acc_mi += mi.item()
        loss = - mi
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    acc_mi /= (input_a.size(0) // batch_size)

    print(f'{e+1} {acc_mi / math.log(2):.04f} {entropy(classes) / math.log(2):.04f}')

    sys.stdout.flush()

######################################################################
# Test

input_a, input_b, classes = create_pairs(train = False)
input_br = input_b[torch.randperm(input_b.size(0))]
acc_mi = 0.0

for batch_a, batch_b, batch_br in zip(input_a.split(batch_size),input_b.split(batch_size),input_br.split(batch_size)):
    mi = model(batch_a, batch_b).mean() - model(batch_a, batch_br).exp().mean().log()
    acc_mi += mi.item()

acc_mi /= (input_a.size(0) // args.batch_size)

print(f'test {acc_mi / math.log(2):.04f} {entropy(classes) / math.log(2):.04f}')

######################################################################

nb_parameters 78097
1 0.8022 1.5829
2 1.3375 1.5829
3 1.3859 1.5829
4 1.4707 1.5829
5 1.4897 1.5829
6 1.5304 1.5829
7 1.5546 1.5829
8 1.5434 1.5829
9 1.5904 1.5829
10 1.5798 1.5829
11 1.5826 1.5829
12 1.6215 1.5829
13 1.6321 1.5829
14 1.6036 1.5829
15 1.5999 1.5829
16 1.5914 1.5829
17 1.6150 1.5829
18 1.6533 1.5829
19 1.6242 1.5829
20 1.6444 1.5829
21 1.6525 1.5829
22 1.6561 1.5829
23 1.6323 1.5829
24 1.6690 1.5829
25 1.6639 1.5829
26 1.6546 1.5829
27 1.6375 1.5829
28 1.6607 1.5829
29 1.7044 1.5829
30 1.6648 1.5829
31 1.6305 1.5829
32 1.6676 1.5829
33 1.6546 1.5829


KeyboardInterrupt: 

Code: https://fleuret.org/files/mi_estimator.py